<a href="https://colab.research.google.com/github/anslava00/PySpark_sber/blob/main/PySpark_lesson4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
google_drive = '/content/drive/MyDrive/Colab_Notebooks/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = spark.read.csv(google_drive + 'Train_Set_90621.csv',  header=True, inferSchema=True).drop(F.col('Amount Defaulted'))

In [ ]:
data.count()

195118

In [ ]:
data.show(3)

+--------------+--------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+--------------+
|Application ID|   Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|Default_Status|
+--------------+------

In [ ]:
data.describe().show()

+-------+-----------------+---------------+---------+------------------+-------------------+--------------------+---------------------------+-----------------+-------------------+------------------+-----------+------------------+------------------+----------------------+-----------------+------------------+------------------------------+-----------------------+----------------------------+--------------------+-------------------+------------------+-----------------+--------------------+------------------+----------------+--------------------+-------------------+
|summary|   Application ID|    Bank Masked|Bank Type|Approved_Timestamp|        Name Masked|Business Owner State|Business_Industry_Type_Code|    Approved_Year|       New Business|              Term|  BankState|     Interest Rate|         Employees|Gross Disbursed Amount|       Term_years|     Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|  Gross_Apprv_Amount|Company Branch Cod

In [ ]:
print(len(data.columns))
data.printSchema()

28
root
 |-- Application ID: integer (nullable = true)
 |-- Bank Masked: string (nullable = true)
 |-- Bank Type: string (nullable = true)
 |-- Approved_Timestamp: string (nullable = true)
 |-- Name Masked: string (nullable = true)
 |-- Business Owner State: string (nullable = true)
 |-- Business_Industry_Type_Code: integer (nullable = true)
 |-- Approved_Year: integer (nullable = true)
 |-- New Business: integer (nullable = true)
 |-- Term: integer (nullable = true)
 |-- BankState: string (nullable = true)
 |-- Interest Rate: integer (nullable = true)
 |-- Employees: integer (nullable = true)
 |-- Gross Disbursed Amount: integer (nullable = true)
 |-- Term_years: integer (nullable = true)
 |-- Jobs Retained: integer (nullable = true)
 |-- Male to Female Employees Ratio: integer (nullable = true)
 |-- Expected Company Income: long (nullable = true)
 |-- Funds available with company: integer (nullable = true)
 |-- Gross_Apprv_Amount: integer (nullable = true)
 |-- Company Branch Code: i

In [ ]:
def check_nan(data):
    data.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in data.columns]).show()

In [ ]:
data_not_nan = data.dropna()
check_nan(data_not_nan)

+--------------+-----------+---------+------------------+-----------+--------------------+---------------------------+-------------+------------+----+---------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+--------------+
|Application ID|Bank Masked|Bank Type|Approved_Timestamp|Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|Default_Status|
+--------------+-----------+---------+----------------

In [ ]:
print(data.count())
print(data_not_nan.count()) 

195118
193042


In [ ]:
data_not_nan.select(F.col('Default_Status'))\
    .groupBy('Default_Status')\
    .count()\
    .withColumn('count', F.col('count'))\
    .show()

+--------------+------+
|Default_Status| count|
+--------------+------+
|             1| 34869|
|             0|158173|
+--------------+------+



In [ ]:
data_not_nan.show(10)

+--------------+---------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+--------------+
|Application ID|    Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|Default_Status|
+--------------+----

In [ ]:
data_not_nan.show(3)

+--------------+--------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+--------------+
|Application ID|   Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|Default_Status|
+--------------+------

In [ ]:
def count_cat(col):
    data_not_nan.select(F.col(col))\
        .groupBy(col)\
        .count()\
        .withColumn('count', F.col('count'))\
        .show()
def count_cat_val(col):
    return data_not_nan.select(F.col(col))\
        .groupBy(col)\
        .count()\
        .withColumn('count', F.col('count'))\
        .count()

In [ ]:
cat_col = ['Bank Masked', 'Bank Type', 'Name Masked', 'Business Owner State', 'BankState', 'Carry-forward Credit', 'Documents Provided']

In [ ]:
for col in cat_col:
    count_cat(col)

+----------------+-----+
|     Bank Masked|count|
+----------------+-----+
|  Bank_Name_1909|    3|
| Bank_Name_51514|    1|
| Bank_Name_27474|    1|
|  Bank_Name_1658|   18|
|   Bank_Name_697|   27|
|   Bank_Name_806|   12|
|  Bank_Name_3559|    2|
|Bank_Number_2607|    2|
|   Bank_Name_671|    6|
| Bank_Name_17580|    1|
|   Bank_Name_153|   65|
| Bank_Name_24264|    1|
| Bank_Number_530|    8|
| Bank_Name_47837|    1|
|   Bank_Name_674|   36|
|  Bank_Name_9004|    1|
| Bank_Number_347|    4|
|  Bank_Name_8683|    1|
| Bank_Name_33843|    1|
| Bank_Name_43255|    1|
+----------------+-----+
only showing top 20 rows

+---------+------+
|Bank Type| count|
+---------+------+
|     Govt| 64288|
|  Private|128754|
+---------+------+

+--------------------+-----+
|         Name Masked|count|
+--------------------+-----+
|Business_Name_140550|    1|
|Business_Name_139609|    1|
| Business_Name_73472|    1|
| Business_Name_62132|    2|
| Business_Name_28654|    1|
|Business_Name_105970|    1

In [ ]:
data_df = data_not_nan.drop('Name Masked', 'Bank Masked', 'Application ID')

In [ ]:
data_df.show(5)

+---------+------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+--------------+
|Bank Type|Approved_Timestamp|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|Default_Status|
+---------+------------------+--------------------+---------------------------+-------------+------------+----+-------------

In [ ]:
for col in data_df.columns:
    print(col, count_cat_val(col))

Bank Type 2
Approved_Timestamp 6408
Business Owner State 10
Business_Industry_Type_Code 2073
Approved_Year 18
New Business 3
Term 363
BankState 12
Interest Rate 11
Employees 430
Gross Disbursed Amount 36847
Term_years 39
Jobs Retained 233
Male to Female Employees Ratio 50
Expected Company Income 1846
Funds available with company 185045
Gross_Apprv_Amount 8990
Company Branch Code 1624
City or Rural 3
Jobs Generated 154
Carry-forward Credit 2
Documents Provided 2
Balance Left 3
Final_Appved_Amount 15389
Default_Status 2


In [ ]:
from pyspark.ml.feature import StringIndexer, IndexToString, OneHotEncoder

In [ ]:
data_df.show(3)

+---------+------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+-------------------+--------------+
|Bank Type|Approved_Timestamp|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Final_Appved_Amount|Default_Status|
+---------+------------------+--------------------+---------------------------+-------------+------------+----+-------------

In [ ]:
from pyspark.ml.functions import vector_to_array

In [ ]:
cat_col = ['Bank Type', 'Business Owner State', 'BankState', 'Carry-forward Credit', 'Documents Provided']
cat_to_one_hot = ['Business Owner State']

In [ ]:
def conv_label_in(col, data):
    
    indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
    indexer = indexer.fit(data)
    indexer_transform = indexer.transform(data).drop(col)
    return (indexer_transform, indexer) if col in cat_to_one_hot else (indexer_transform, 0)

def conv_one_hot(col, data, label):
    ohe_encoder = OneHotEncoder(inputCols=[col+"_index"], outputCols=[col+"_onehot"])
    ohe_encoder = ohe_encoder.fit(data)
    ohe_encoder = ohe_encoder.transform(data).drop(col)

    df_col_onehot = ohe_encoder.select('*', vector_to_array(col+"_onehot").alias(col+'_onehot_arr'))
    num_categories = len(df_col_onehot.first()[col+"_onehot_arr"]) 
    cols_expanded = [(F.col(col+'_onehot_arr')[i].alias(f'{label[i]}'))  for i in range(num_categories)]
    df_cols_onehot = df_col_onehot.select('*', *cols_expanded)
    return df_cols_onehot.drop(col+'_onehot', col+'_onehot_arr')


In [ ]:
data_label_cod = data_df
label_list = []
for col in cat_col:
    data_label_cod, label = conv_label_in(col, data_label_cod)
    if label:
        label_list.append(label)
data_label_cod.show(3)

+------------------+---------------------------+-------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+------------+-------------------+--------------+---------------+--------------------------+---------------+--------------------------+------------------------+
|Approved_Timestamp|Business_Industry_Type_Code|Approved_Year|New Business|Term|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Balance Left|Final_Appved_Amount|Default_Status|Bank Type_index|Business Owner State_index|BankState_index|Carry-forward Credit_index|Documents Provided_index|
+------------------+---------------------------+-------------+----------

In [ ]:
data.select('Business Owner State', 'BankState').show()

+--------------------+-------------+
|Business Owner State|    BankState|
+--------------------+-------------+
|       Uttar Pradesh|Uttar Pradesh|
|       Uttar Pradesh|Uttar Pradesh|
|       Uttar Pradesh|  Maharashtra|
|              Kerala|        Delhi|
|               Delhi|  Maharashtra|
|         Maharashtra|  Maharashtra|
|               Delhi|        Delhi|
|       Uttar Pradesh|        Delhi|
|             Haryana|      Haryana|
|         Maharashtra|        Delhi|
|       Uttar Pradesh|Uttar Pradesh|
|             Haryana|        Delhi|
|               Delhi|        Delhi|
|         Maharashtra|  Maharashtra|
|              Kerala|       Kerala|
|       Uttar Pradesh|Uttar Pradesh|
|               Delhi|        Delhi|
|         Maharashtra|      Haryana|
|             Haryana|      Haryana|
|              Punjab|       Punjab|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
label_list[0].labels

['Delhi',
 'Haryana',
 'Maharashtra',
 'Uttar Pradesh',
 'Orissa',
 'Tamil Nadu',
 'Uttarakhand',
 'Karnataka',
 'Punjab',
 'Kerala']

In [ ]:
data_hot_cod = data_label_cod
for index, col in enumerate(cat_to_one_hot):
    data_hot_cod = conv_one_hot(col, data_hot_cod, label_list[index].labels)
data_hot_cod.show(3)

+------------------+---------------------------+-------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+------------+-------------------+--------------+---------------+--------------------------+---------------+--------------------------+------------------------+-----+-------+-----------+-------------+------+----------+-----------+---------+------+
|Approved_Timestamp|Business_Industry_Type_Code|Approved_Year|New Business|Term|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Balance Left|Final_Appved_Amount|Default_Status|Bank Type_index|Business Owner State_index|BankState_index|Carry-forward Credit_index|Documents P

In [ ]:
data_hot_cod.count()

193042

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
train, test = data_not_nan.randomSplit([0.7, 0.3], seed=7)
Bank_Type_indexer = StringIndexer(inputCol='Bank Type', outputCol="bank_type_index")
BOS_indexer = StringIndexer(inputCol='Business Owner State', outputCol="business_owner_state_index")
CFC_indexer = StringIndexer(inputCol='Carry-forward Credit', outputCol="carry_forward_credit_index")
DP_indexer = StringIndexer(inputCol='Documents Provided', outputCol="documents_provided_index")



ohe_encoder = OneHotEncoder(inputCols=["bank_type_index", "business_owner_state_index"],
                        outputCols=["bank_type_vec", "business_owner_state_vec"])

feature_columns = [
                'carry_forward_credit_index',
                'documents_provided_index',
                'Business_Industry_Type_Code',
                'Approved_Year',
                'New Business',
                'Term',
                # 'BankState',
                'Interest Rate',
                'Employees',
                'Gross Disbursed Amount',
                'Term_years',
                'Jobs Retained',
                'Male to Female Employees Ratio',
                'Expected Company Income',
                'Funds available with company',
                'Gross_Apprv_Amount',
                'Company Branch Code',
                'City or Rural',
                'Jobs Generated',
                # 'Balance Left',
                'Final_Appved_Amount',
                'bank_type_vec',
                'business_owner_state_vec'                 
]

target = 'Default_Status'

vector_assembler = VectorAssembler(inputCols = feature_columns, outputCol = 'features')


lr = LogisticRegression(featuresCol='features', labelCol=target, predictionCol='prediction',
                        maxIter=10,
                        elasticNetParam = 1,
                        probabilityCol='proba')

pipeline = Pipeline(stages=[
                        Bank_Type_indexer,
                        BOS_indexer,
                        CFC_indexer,
                        DP_indexer,
                        ohe_encoder,
                        vector_assembler,
                        lr
])


Model_best_gip_par = pipeline.fit(train)

In [ ]:
# transformed_data.select('prediction').groupBy('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|135064|
|       1.0|   102|
+----------+------+



In [ ]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [10, 20, 200, 100, 500]) \
    .addGrid(lr.regParam, [0, 0.1, 0.2, 0.3, 1])\
    .addGrid(lr.elasticNetParam, [0, 0.3, 0.7, 1])\
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                                                  labelCol=target, metricName='areaUnderROC'),
                          numFolds=6,
                          parallelism=2)
Model = crossval.fit(train)

In [ ]:
Model.avgMetrics

[0.7107644984860285,
 0.7107646390142125,
 0.7107650266409345,
 0.7107651475344343,
 0.7005391299162586,
 0.7048369262449695,
 0.7031443783049843,
 0.5,
 0.6945263985181335,
 0.7031443783049843,
 0.5,
 0.5,
 0.6908178658818055,
 0.5,
 0.5,
 0.5,
 0.6823872836322215,
 0.5,
 0.5,
 0.5,
 0.7107309360902966,
 0.7107340234949374,
 0.7107308450637458,
 0.7107308537616924,
 0.7005374965397722,
 0.7048369262449695,
 0.7031443783049843,
 0.5,
 0.6945256732599576,
 0.7031443783049843,
 0.5,
 0.5,
 0.6908182187711644,
 0.5,
 0.5,
 0.5,
 0.6823877254900887,
 0.5,
 0.5,
 0.5,
 0.7105415262007412,
 0.710541467928635,
 0.7105438925686386,
 0.710541290647451,
 0.7005376440980889,
 0.7048369262449695,
 0.7031443783049843,
 0.5,
 0.6945252453138663,
 0.7031443783049843,
 0.5,
 0.5,
 0.690817267121561,
 0.5,
 0.5,
 0.5,
 0.6823874366506462,
 0.5,
 0.5,
 0.5,
 0.7105193064852166,
 0.7105202947167241,
 0.7105188304099173,
 0.7105173949224838,
 0.700539617650974,
 0.7048369262449694,
 0.7031443783049843,
 0

In [ ]:
import numpy as np
Model.getEstimatorParamMaps()[np.argmax(Model.avgMetrics)]

{Param(parent='LogisticRegression_66078dd293d6', name='maxIter', doc='max number of iterations (>= 0).'): 10,
 Param(parent='LogisticRegression_66078dd293d6', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
 Param(parent='LogisticRegression_66078dd293d6', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0}

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                          labelCol=target, 
                                          metricName='areaUnderROC')
evaluator.evaluate(Model.transform(test))

0.7127749980696461

In [ ]:
transformed_data = Model_best_gip_par.transform(train)

evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                          labelCol=target, 
                                          metricName='areaUnderROC')
evaluator.evaluate(transformed_data)

0.7109436303848229

In [ ]:
transformed_data = Model_best_gip_par.transform(test)

evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                          labelCol=target, 
                                          metricName='areaUnderROC')
evaluator.evaluate(transformed_data)

0.7121382976826156